In [16]:
# pip install scikit-learn

In [17]:
from sklearn import datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [18]:
# Read your CSV file
data = pd.read_csv('final_data.csv')
titles = data['Title']
data = pd.get_dummies(data.drop(columns = "Title"))
print(data.head())

KeyboardInterrupt: 

In [ ]:
# Select training data (drop the 'Title' column)
X = data.drop(columns = ["liked"])
y = data["liked"]

In [ ]:
# # Suppose train_data and test_data are already split

# # Apply one-hot encoding
# train_data = pd.get_dummies(train_data)
# test_data = pd.get_dummies(test_data)

# # Align the columns between train and test (VERY IMPORTANT)
# train_data, test_data = train_data.align(test_data, join='left', axis=1, fill_value=0)


In [15]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=446)

# Scale features (important for SVMs!)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

svm_model = SVC(kernel='linear')


svm_model.fit(X_train, y_train)


predictions = svm_model.predict(X_test)

In [21]:
print(predictions)

[1 1 0 ... 1 1 1]


In [19]:

from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_test, predictions))

Accuracy: 0.9012765957446809
